In [20]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader
import torchvision
from torchvision.models.detection.image_list import ImageList
from torchsummary import summary
import io
import sys

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
DEVICE

device(type='cpu')

In [4]:
IMAGE_DIR = '../dataset/images'
ANNOTATIONS_DIR = '../dataset/annotations'
TARGET_SIZE = (224, 224)
BATCH_SIZE = 8

In [5]:
class_names = [d for d in os.listdir(IMAGE_DIR) if os.path.isdir(os.path.join(IMAGE_DIR, d))]

In [6]:
class_names

['motorcycle', 'airplane', 'face']

In [7]:
csv_files = [f for f in os.listdir(ANNOTATIONS_DIR) if f.endswith('.csv')]

In [8]:
label_map = {name: i + 1 for i, name in enumerate(class_names)}

In [9]:
label_map

{'motorcycle': 1, 'airplane': 2, 'face': 3}

In [10]:
dataset = []

for i in range(len(class_names)):
    class_name = class_names[i]
    class_dir = os.path.join(IMAGE_DIR, class_name)
    csv_file_name = csv_files[i]
    
    csv_path = os.path.join(ANNOTATIONS_DIR, csv_file_name)
    df_annotations = pd.read_csv(csv_path)

    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        
        image = cv2.imread(image_path)
        if image is None:
            continue
                
        h, w, _ = image.shape

        row = df_annotations[df_annotations['image_name'] == image_name]

        # ---- FIX HERE ----
        if row.empty:
            continue
        # -------------------

        ann = row.iloc[0, 1:].tolist()

        ann[0] = (ann[0] / w) * 224
        ann[1] = (ann[1] / h) * 224
        ann[2] = (ann[2] / w) * 224
        ann[3] = (ann[3] / h) * 224

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        
        image_tensor = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1) / 255.0
        label_tensor = torch.tensor([class_names.index(class_name)], dtype=torch.int64)
        ann_tensor = torch.tensor([ann], dtype=torch.float32)

        target = {
            'boxes': ann_tensor,
            'labels': label_tensor
        }
        dataset.append((image_tensor, target))

def collate_fn(batch):
    return tuple(zip(*batch))

dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)


In [11]:
resnet_model = torchvision.models.resnet18()

In [ ]:
summary(resnet_model,(3,224,224))

In [14]:
backbone = torch.nn.Sequential(*list(resnet_model.children())[:-2])
backbone.out_channels = 512 # custom attribute
backbone.out_channels

512

In [19]:
output_dir = "model_output"
os.makedirs(output_dir, exist_ok=True)

# file path
save_path = os.path.join(output_dir, "model_summary.txt")

# capture printed summary
buffer = io.StringIO()
sys_stdout = sys.stdout  # save real stdout
sys.stdout = buffer      # redirect stdout

summary(backbone, (3, 224, 224))   # <-- prints inside buffer
summary(backbone,(3,224,224))

sys.stdout = sys_stdout  # restore real stdout

# write to file
with open(save_path, "w") as f:
    f.write(buffer.getvalue())

print("Model summary saved to:", save_path)

NameError: name 'sys' is not defined